In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786_dataset/

Mounted at /content/drive
/content/drive/MyDrive/ECE1786_dataset


In [ ]:
!ls

final_dataset_suicide.csv
final_dataset_suicide_test.csv
final_dataset_suicide_test_huggingface.csv
final_dataset_suicide_test_openai.csv
final_dataset_suicide_train.csv
final_dataset_suicide_train_huggingface.csv
final_dataset_suicide_train_openai.csv
openai_apikey.txt


In [2]:
!pip install openai
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 15.3 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=1aa11cbf9d42ddb3be400239515c75484ef560bf7294f1333145ce285bce2cc2
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.6 MB/s 
     |████████████████████████████████| 182 kB 53.6 MB/s 
     |████████████████████████████████| 7.6 MB 38.0 MB/s 


In [3]:
import openai
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("./final_dataset_suicide.csv")
X, y = df['text'], df['class']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=114514, stratify=y)

In [6]:
print("===== train dataset =====")
print("# of rows: ", y_train.count())
print("# of suicide labels: ", y_train[y_train=="suicide"].count())
print("# of non-suicide labels: ", y_train[y_train=="non-suicide"].count())
print("===== validation dataset =====")
print("# of rows: ", y_test.count())
print("# of suicide labels: ", y_test[y_test=="suicide"].count())
print("# of non-suicide labels: ", y_test[y_test=="non-suicide"].count())

===== train dataset =====
# of rows:  176000
# of suicide labels:  88000
# of non-suicide labels:  88000
===== validation dataset =====
# of rows:  44000
# of suicide labels:  22000
# of non-suicide labels:  22000


In [7]:
train_df = pd.DataFrame({'text': X_train, 'class': y_train}).reset_index()
test_df = pd.DataFrame({'text': X_test, 'class': y_test}).reset_index()

In [ ]:
train_df.to_csv("./final_dataset_suicide_train.csv", index=False, index_label=None)
test_df.to_csv("./final_dataset_suicide_test.csv", index=False, index_label=None)

## openai dataset

In [8]:
train_df.rename(columns={"text": "prompt", "class": "completion"}, inplace=True)
test_df.rename(columns={"text": "prompt", "class": "completion"}, inplace=True)

### 1/10 openai

In [12]:
X_train_10, _, y_train_10, _ = train_test_split(train_df.prompt, train_df.completion, test_size=0.9, random_state=1919819, stratify=train_df.completion)

In [13]:
_, X_test_10, _, y_test_10 = train_test_split(test_df.prompt, test_df.completion, test_size=0.1, random_state=1919819, stratify=test_df.completion)

In [14]:
train_10_df = pd.DataFrame({'prompt': X_train_10, 'completion': y_train_10}).reset_index()
test_10_df = pd.DataFrame({'prompt': X_test_10, 'completion': y_test_10}).reset_index()

In [15]:
train_10_df.to_csv("final_dataset_suicide_train_10_openai.csv", index=False, index_label=None)
test_10_df.to_csv("final_dataset_suicide_test_10_openai.csv", index=False, index_label=None)

In [16]:
!openai tools fine_tunes.prepare_data -f "final_dataset_suicide_train_10_openai.csv"
!openai tools fine_tunes.prepare_data -f "final_dataset_suicide_test_10_openai.csv"

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 17600 prompt-completion pairs
- The input file should contain exactly two columns/keys per row. Additional columns/keys present are: ['index']
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 35 examples that are very long. These are rows: [627, 1269, 1597, 1602, 2804, 3250, 3428, 4176, 4394, 4780, 5528, 5614, 5870, 5956, 7202, 7753, 8464, 8600, 9545, 9773, 10050, 10625, 11503, 11711, 12472, 12494, 12921, 13794, 14122, 15537, 16597, 16651, 16714, 16911, 16984]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at

### openai

In [ ]:
train_df.to_csv("final_dataset_suicide_train_openai.csv", index=False, index_label=None)
test_df.to_csv("final_dataset_suicide_test_openai.csv", index=False, index_label=None)

In [ ]:
!openai tools fine_tunes.prepare_data -f "final_dataset_suicide_train_openai.csv"
!openai tools fine_tunes.prepare_data -f "final_dataset_suicide_test_openai.csv"

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 176000 prompt-completion pairs
- The input file should contain exactly two columns/keys per row. Additional columns/keys present are: ['index']
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 287 examples that are very long. These are rows: [399, 495, 848, 1677, 1933, 2222, 2490, 2599, 2764, 2951, 4457, 4538, 5709, 5883, 5898, 6048, 6080, 7029, 8926, 9107, 9444, 9485, 11028, 11893, 12414, 14365, 15759, 16923, 17183, 17349, 17663, 18915, 18976, 20161, 20652, 22751, 22794, 25060, 25714, 27099, 27143, 28313, 29061, 29315, 29591, 31084, 31563, 31699, 32348, 32922, 33202, 33992, 34757, 35019, 35222, 35578, 35826, 

### hugging face

In [ ]:
train_df.drop([399, 495, 848, 1677, 1933, 2222, 2490, 2599, 2764, 2951, 4457, 4538, 5709, 5883, 5898, 6048, 6080, 7029, 8926, 9107, 9444, 9485, 11028, 11893, 12414, 14365, 15759, 16923, 17183, 17349, 17663, 18915, 18976, 20161, 20652, 22751, 22794, 25060, 25714, 27099, 27143, 28313, 29061, 29315, 29591, 31084, 31563, 31699, 32348, 32922, 33202, 33992, 34757, 35019, 35222, 35578, 35826, 36646, 37224, 37524, 37947, 38256, 39903, 40980, 41153, 41707, 42185, 42425, 42435, 43247, 43977, 44178, 44505, 44913, 45026, 45733, 47375, 47742, 48318, 48660, 49631, 50133, 51648, 52099, 53918, 54829, 55395, 55425, 55642, 55709, 56966, 58673, 59119, 59486, 59655, 59835, 59959, 60250, 60516, 60953, 61577, 61730, 62970, 63273, 64005, 64553, 65470, 66100, 66267, 66598, 67066, 67562, 68523, 68933, 69362, 70566, 70902, 71208, 72604, 72682, 76750, 76755, 77139, 77158, 77620, 77726, 77892, 78924, 79125, 79394, 79622, 80928, 81151, 81156, 81979, 83579, 84462, 85057, 85352, 87953, 88909, 92328, 92703, 94461, 94660, 96923, 98013, 98040, 98441, 98992, 100162, 100314, 100782, 101264, 101895, 102422, 102577, 103220, 104066, 104852, 105368, 105449, 106898, 107215, 107896, 107923, 108668, 109053, 109072, 110469, 110548, 110555, 110860, 110914, 111315, 111481, 111548, 111924, 112469, 112592, 112728, 112751, 114079, 114598, 114834, 115539, 116027, 117103, 117184, 117418, 117426, 117523, 118031, 118310, 118392, 118495, 118800, 118931, 118963, 119250, 119302, 119681, 120416, 123031, 123186, 123463, 124906, 124924, 125341, 126450, 128194, 129543, 129651, 131805, 132193, 133294, 133487, 133691, 133731, 133884, 134161, 134239, 134797, 136116, 136181, 136459, 137559, 138011, 138401, 138825, 138902, 142591, 142768, 143314, 143509, 144235, 145068, 145114, 145376, 146474, 146553, 147073, 147534, 148147, 149300, 149450, 150969, 151082, 151794, 152108, 152961, 154961, 156054, 156991, 157398, 158641, 158945, 159604, 159707, 160291, 160976, 161065, 161431, 161894, 162372, 163945, 163975, 164008, 165268, 165885, 166043, 167421, 167778, 167969, 168190, 168382, 169343, 169346, 169611, 172018, 172082, 172395, 172553, 172780, 173718, 173932, 174022], axis=0, inplace=True)

In [ ]:
test_df.drop([20, 366, 1018, 1168, 2684, 2790, 3605, 5771, 7138, 9639, 10233, 10530, 10721, 10736, 11683, 12082, 12333, 12483, 13920, 15542, 15611, 16683, 16791, 16956, 18034, 18416, 19145, 19240, 19616, 19791, 19917, 20307, 21180, 22447, 22554, 22610, 23380, 23610, 23651, 24131, 25232, 25611, 27027, 27940, 29558, 30112, 30540, 31503, 32925, 34123, 34736, 34860, 34873, 36802, 37557, 37618, 38040, 38479, 38934, 39256, 39409, 41262, 41979, 42308, 42533, 42827, 43271], axis=0, inplace=True)

In [ ]:
train_df.to_csv("final_dataset_suicide_train_huggingface.csv", index=False, index_label=None)
test_df.to_csv("final_dataset_suicide_test_huggingface.csv", index=False, index_label=None)